In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta
import matplotlib as mpl
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
from datetime import datetime, timedelta
import warnings
import datetime
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)
mpl.style.use('seaborn')
pd.set_option('display.max_columns', 1000)
from sklearn.metrics import accuracy_score


In [2]:
df=pd.read_csv("aud_usd_h4_h1_all_merged_corr_0.01.csv")
df.columns = df.columns.str.replace('.','_')

In [3]:
df.head()

Unnamed: 0           time_local  target1_x  target2_x  target3_x  \
0           0  2010-01-05 18:00:00     0.9117     0.9135     0.9142   
1           1  2010-01-05 22:00:00     0.9135     0.9142     0.9142   
2           2  2010-01-06 02:00:00     0.9142     0.9142     0.9156   
3           3  2010-01-06 06:00:00     0.9142     0.9156     0.9184   
4           4  2010-01-06 10:00:00     0.9156     0.9184     0.9196   

   bid_close_x  Volatility_USD_hourly  PC_macro_economic_ind_USD_hourly  \
0       0.9124                    2.0                         -2.477564   
1       0.9117                    2.0                         -2.477564   
2       0.9135                    3.0                          0.000000   
3       0.9142                    3.0                          0.000000   
4       0.9142                    0.0                          0.000000   

   Volatility_USD_daily  PC_macro_economic_ind_USD_daily  \
0                   2.0                        -2.477564   
1                   2.0                        -2.477564   
2                   1.5                        -0.395704   
3                   1.5                        -0.395704   
4                   1.5                        -0.395704   

   Volatility_AUD_hourly  PC_macro_economic_ind_AUD_hourly  \
0                    0.0                          0.000000   
1                    0.0                          0.000000   
2                    0.5                         -1.543434   
3                    0.5                         -1.543434   
4                    0.5                         -1.543434   

   PC_macro_economic_ind_AUD_daily  ANZ Job Advertisements_PC_AUD_hourly  \
0                        -1.050000                                   0.0   
1                        -1.050000                                   0.0   
2                        -1.543434                                   0.0   
3                        -1.543434                                   0.0   
4                        -1.543434                                   0.0   

   Building Permits (MoM)_PC_AUD_hourly  Building Permits (YoY)_PC_AUD_hourly  \
0                              0.000000                              0.000000   
1                              0.000000                              0.000000   
2                             -4.484536                              1.397668   
3                             -4.484536                              1.397668   
4                             -4.484536                              1.397668   

   CFTC AUD NC net positions_PC_AUD_hourly  \
0                                      0.0   
1                                      0.0   
2                                      0.0   
3                                      0.0   
4                                      0.0   

   Consumer Price Index (QoQ)_PC_AUD_hourly  \
0                                       0.0   
1                                       0.0   
2                                       0.0   
3                                       0.0   
4                                       0.0   

   Dwelling Unit Starts (QoQ)_PC_AUD_hourly  \
0                                       0.0   
1                                       0.0   
2                                       0.0   
3                                       0.0   
4                                       0.0   

   Employment Change s_a__PC_AUD_hourly  \
0                                   0.0   
1                                   0.0   
2                                   0.0   
3                                   0.0   
4                                   0.0   

   Export Price Index (QoQ)_PC_AUD_hourly  Exports_PC_AUD_hourly  \
0                                     0.0                    0.0   
1                                     0.0                    0.0   
2                                     0.0                    0.0   
3                                     0.0                    0.0   
4    

In [4]:
df=df.dropna(how='all',axis=1)
df.shape

(14028, 1064)

In [5]:
df['year']=pd.DatetimeIndex(df['time_local']).year
df['month']=pd.DatetimeIndex(df['time_local']).month
df['day']=pd.DatetimeIndex(df['time_local']).day
df['hour']=pd.DatetimeIndex(df['time_local']).hour
df['dayofweek']=pd.DatetimeIndex(df['time_local']).dayofweek



In [6]:
import datetime   
#date = datetime.date.today()
date=pd.datetime.now().strftime("%Y-%m-%d %I:%M:%S")
date

'2019-04-17 10:27:11'

In [7]:
df['today']=date
df['today'] =pd.DatetimeIndex(df['today'])

In [8]:
df['nb_months'] = ((df['today'] - pd.to_datetime(df['time_local'] ))/np.timedelta64(1, 'M'))
df['nb_months'] = df['nb_months'].astype(int)

In [9]:
col_drop= df.filter(regex='time').columns

In [10]:
target_cols = df.filter(regex='target').columns

In [11]:
cols_x = df.filter(regex='_x').columns

In [12]:
cols_y = df.filter(regex='_y').columns

In [13]:
X=df.drop(['Unnamed: 0','time_local', 'target2_x', 'target3_x', 'target1_y', 'target2_y',
       'target3_y', 'class_target2', 'class_target3',
       'PC_AUD_H4_target1', 'bid_close_y', 'today','target1_x'],axis=1)

In [14]:
X=X.fillna(0)

In [15]:
df_temp=X.select_dtypes(include=['object'])

In [16]:
obj=df_temp.loc[:, df_temp.dtypes =='object']
for col in obj:
    obj[col] = pd.to_numeric(obj[col], errors='coerce')
df_temp[obj.columns]=obj

In [17]:
df_temp=df_temp.replace([np.inf, -np.inf], np.nan)
df_temp=df_temp.fillna(0)
X[df_temp.columns]=df_temp

In [18]:
X_final= X[X['year'] < 2018]
s=X[['year','month', 'dayofweek']]
for each in s:
    dummies = pd.get_dummies(s[each], prefix=each, drop_first=False)
    X_final=X_final.drop([each],axis=1)
    X_final = pd.concat([X_final, dummies], axis=1)

In [19]:
X_final=X_final[X_final.replace([np.inf, -np.inf], np.nan).notnull().all(axis=1)]

In [20]:
all_col=X_final.columns
drop_col=list(x for x in all_col if 'h1_5' in x or 'h1_6' in x or 'h1_7' in x  )
X_final=X_final.drop(columns=drop_col,axis=1)

In [21]:
y=X_final['class_target1']
X1=X_final.drop(['class_target1'],axis=1)

In [22]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(X1)
X1=scaler.transform(X1)

In [23]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
np.seed=0
from sklearn.linear_model import LogisticRegression

In [24]:
from imblearn.over_sampling import SMOTE
smt = SMOTE(random_state=1)
X2, y1 = smt.fit_sample(X1, y)
len(X2),len(y1)

(20976, 20976)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X2, y1, test_size=0.3,random_state=1,stratify=y1)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)


(14683, 1057) (14683,)
(6293, 1057) (6293,)


In [27]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier()
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)

In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

        -1.0       0.58      0.73      0.65      2097
         0.0       0.88      0.92      0.90      2098
         1.0       0.57      0.40      0.47      2098

   micro avg       0.69      0.69      0.69      6293
   macro avg       0.68      0.69      0.68      6293
weighted avg       0.68      0.69      0.68      6293



In [30]:
print(classification_report(y_train,gb.predict(X_train)))

              precision    recall  f1-score   support

        -1.0       0.70      0.85      0.77      4895
         0.0       0.89      0.94      0.91      4894
         1.0       0.76      0.55      0.64      4894

   micro avg       0.78      0.78      0.78     14683
   macro avg       0.78      0.78      0.77     14683
weighted avg       0.78      0.78      0.77     14683



In [31]:
accuracy_score(y_test,y_pred)

0.6861592245351978

In [32]:
accuracy_score(y_train,gb.predict(X_train))

0.7802901314445276

In [34]:
pd.crosstab(y_test,y_pred)

col_0  -1.0   0.0   1.0
row_0                  
-1.0    884   725   515
 0.0    877   739   474
 1.0    855   729   495

In [35]:
pd.crosstab(y_train,gb.predict(X_train))

col_0  -1.0   0.0   1.0
row_0                  
-1.0   3969     6   893
 0.0    177  4580   145
 1.0   1833   581  2499

In [36]:
gb

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [37]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.8,n_estimators=80)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)

In [38]:
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.5801684411250596, 0.6032827078934823)

In [45]:

from sklearn.model_selection import GridSearchCV
parameters = {
    "learning_rate": [0.1],
    "max_depth":[3,5,8],
    "criterion": ["friedman_mse"],
    "n_estimators":[90,100,120],
    
    "subsample":[0.9,1.0]
}

clf = GridSearchCV(gb, parameters, cv=5,verbose=5)
clf.fit(X_train,y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.9 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.9, score=0.6729067392784207, total= 8.8min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.9 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  8.8min remaining:    0.0s


[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.9, score=0.6705241660993874, total= 9.4min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.9 


...

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 26.8min remaining:    0.0s


[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.9, score=0.6889267461669506, total= 9.4min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.9 


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 36.2min remaining:    0.0s


[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.9, score=0.6821124361158433, total= 9.6min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=1.0 
[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=1.0, score=0.6684819605173588, total= 9.9min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=1.0 
[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=1.0, score=0.6756296800544588, total=30.8min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=1.0 
[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=1.0, score=0.6785835886959483, total=10.7min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=1.0 
[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=1.0

KeyboardInterrupt: 

In [46]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.8,n_estimators=90)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.5855712696647069, 0.606075052782129)

In [47]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.8,n_estimators=100)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.5865247099952328, 0.6105700469931213)

In [48]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.8,n_estimators=110)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.5893850309868107, 0.6132261799359804)

In [49]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.8,n_estimators=110,max_depth=4)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.6218020022246941, 0.6592658176122046)

In [51]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.8,n_estimators=120,max_depth=4)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.628476084538376, 0.6679833821426139)

In [52]:
#max_features="auto"
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.85,n_estimators=130,max_depth=4,max_features="auto")
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.6326076593039885, 0.6744534495675271)

# Final_model

In [55]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,model.predict(X_train))

(0.6850468774829175, 0.7499829735067766)

In [56]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

        -1.0       0.59      0.76      0.67      2124
         0.0       0.87      0.91      0.89      2090
         1.0       0.57      0.38      0.45      2079

   micro avg       0.69      0.69      0.69      6293
   macro avg       0.68      0.68      0.67      6293
weighted avg       0.68      0.69      0.67      6293



In [85]:
print(classification_report(y_train,model.predict(X_train)))

              precision    recall  f1-score   support

        -1.0       0.66      0.84      0.74      4868
         0.0       0.88      0.93      0.91      4902
         1.0       0.72      0.48      0.58      4913

   micro avg       0.75      0.75      0.75     14683
   macro avg       0.75      0.75      0.74     14683
weighted avg       0.75      0.75      0.74     14683



In [58]:
pd.crosstab(y_test,y_pred)

col_0  -1.0   0.0   1.0
row_0                  
-1.0   1623     7   494
 0.0     95  1908    87
 1.0   1027   272   780

In [86]:
pd.crosstab(y_train,model.predict(X_train))

col_0  -1.0   0.0   1.0
row_0                  
-1.0   4085    16   767
 0.0    193  4568   141
 1.0   1951   603  2359

In [75]:
y_test1=pd.DataFrame(y_test)
y_pred1=pd.DataFrame(y_pred)
X_test1=pd.DataFrame(X_test)
y_test1.columns=['y_test']
y_pred1.columns=['y_pred']
cols=X_final.drop(['class_target1'],axis=1)
X_test1.columns=cols.columns
y_pred1.index=y_test1.index
df_plot_all=pd.concat([X_test1,y_test1,y_pred1],axis=1)

In [76]:
df_plot_all.head()

bid_close_x  Volatility_USD_hourly  PC_macro_economic_ind_USD_hourly  \
0    -1.075352               1.322736                          0.215520   
1    -1.269892              -1.108923                          0.069308   
2     0.160345               0.106906                          0.069308   
3     0.600050              -1.108923                          0.069308   
4    -0.883894               1.322736                          0.069308   

   Volatility_USD_daily  PC_macro_economic_ind_USD_daily  \
0              0.698319                         0.201521   
1             -2.582012                         0.093277   
2             -0.315258                        -1.732986   
3             -2.849203                         0.133897   
4              0.360460                         1.638483   

   Volatility_AUD_hourly  PC_macro_economic_ind_AUD_hourly  \
0              -0.723057                          0.013671   
1              -0.723057                          0.013671   
2               0.681062                         -0.084736   
3              -0.186492                          0.013747   
4              -0.723057                          0.013671   

   PC_macro_economic_ind_AUD_daily  ANZ Job Advertisements_PC_AUD_hourly  \
0                         0.019957                              0.023481   
1                         0.020036                              0.023481   
2                        -0.044362                              0.023481   
3                         0.016050                              0.023481   
4                         0.005749                              0.023481   

   Building Permits (MoM)_PC_AUD_hourly  Building Permits (YoY)_PC_AUD_hourly  \
0                              0.025981                             -0.005907   
1                              0.025981                             -0.005907   
2                              0.025981                             -0.005907   
3                              0.025981                             -0.005907   
4                              0.025981                             -0.005907   

   CFTC AUD NC net positions_PC_AUD_hourly  \
0                                  0.01347   
1                                  0.01347   
2                                  0.01347   
3                                  0.01347   
4                                  0.01347   

   Consumer Price Index (QoQ)_PC_AUD_hourly  \
0                                 -0.017218   
1                                 -0.017218   
2                                 -0.017218   
3                                 -0.017218   
4                                 -0.017218   

   Dwelling Unit Starts (QoQ)_PC_AUD_hourly  \
0                                   0.02419   
1                                   0.02419   
2                                   0.02419   
3                                   0.02419   
4                                   0.02419   

   Employment Change s_a__PC_AUD_hourly  \
0                             -0.015842   
1                             -0.015842   
2                             -0.015842   
3                             -0.015842   
4                             -0.015842   

   Export Price Index (QoQ)_PC_AUD_hourly  Exports_PC_AUD_hourly  \
0                                0.026826               0.037936   
1                                0.026826               0.037936   
2                                0.026826               0.037936   
3                                0.026826               0.037936   
4                                0.026826               0.037936   

   Fulltime employment_PC_AUD_hourly  HIA New Home Sales (MoM)_PC_AUD_hourly  \
0                          -0.004086                                0.022187   
1                          -0.004086                                0.022187   
2                          -0.004086                                0.022187   
3                          -0.004086       

In [79]:
df_plot_all.head()

bid_close_x  Volatility_USD_hourly  PC_macro_economic_ind_USD_hourly  \
0    -1.075352               1.322736                          0.215520   
1    -1.269892              -1.108923                          0.069308   
2     0.160345               0.106906                          0.069308   
3     0.600050              -1.108923                          0.069308   
4    -0.883894               1.322736                          0.069308   

   Volatility_USD_daily  PC_macro_economic_ind_USD_daily  \
0              0.698319                         0.201521   
1             -2.582012                         0.093277   
2             -0.315258                        -1.732986   
3             -2.849203                         0.133897   
4              0.360460                         1.638483   

   Volatility_AUD_hourly  PC_macro_economic_ind_AUD_hourly  \
0              -0.723057                          0.013671   
1              -0.723057                          0.013671   
2               0.681062                         -0.084736   
3              -0.186492                          0.013747   
4              -0.723057                          0.013671   

   PC_macro_economic_ind_AUD_daily  ANZ Job Advertisements_PC_AUD_hourly  \
0                         0.019957                              0.023481   
1                         0.020036                              0.023481   
2                        -0.044362                              0.023481   
3                         0.016050                              0.023481   
4                         0.005749                              0.023481   

   Building Permits (MoM)_PC_AUD_hourly  Building Permits (YoY)_PC_AUD_hourly  \
0                              0.025981                             -0.005907   
1                              0.025981                             -0.005907   
2                              0.025981                             -0.005907   
3                              0.025981                             -0.005907   
4                              0.025981                             -0.005907   

   CFTC AUD NC net positions_PC_AUD_hourly  \
0                                  0.01347   
1                                  0.01347   
2                                  0.01347   
3                                  0.01347   
4                                  0.01347   

   Consumer Price Index (QoQ)_PC_AUD_hourly  \
0                                 -0.017218   
1                                 -0.017218   
2                                 -0.017218   
3                                 -0.017218   
4                                 -0.017218   

   Dwelling Unit Starts (QoQ)_PC_AUD_hourly  \
0                                   0.02419   
1                                   0.02419   
2                                   0.02419   
3                                   0.02419   
4                                   0.02419   

   Employment Change s_a__PC_AUD_hourly  \
0                             -0.015842   
1                             -0.015842   
2                             -0.015842   
3                             -0.015842   
4                             -0.015842   

   Export Price Index (QoQ)_PC_AUD_hourly  Exports_PC_AUD_hourly  \
0                                0.026826               0.037936   
1                                0.026826               0.037936   
2                                0.026826               0.037936   
3                                0.026826               0.037936   
4                                0.026826               0.037936   

   Fulltime employment_PC_AUD_hourly  HIA New Home Sales (MoM)_PC_AUD_hourly  \
0                          -0.004086                                0.022187   
1                          -0.004086                                0.022187   
2                          -0.004086                                0.022187   
3                          -0.004086       